### Work left to do

##### 1(Abdul). Create a script that queries the Google Books API for each book in the books_data.csv dataset and retrieves the pageCount and averageRating fields from the response and adds them as columns to the enhanced_books_data.csv

##### Expected columns: book_title, page_count, average_rating 

##### 2(Joseph). Sentiment analysis of the review/text to produce a column with either a positive(1) or negative(1) sentiment; said column can be used as a feature in a ML model

##### 3. Four models have been built

###### 1. Linear regression - Joseph draft 
###### 2. K-nearest neighbors - Joseph draft 
###### 3. _ model - Brian 
###### 4. _ model - Griffin 

##### 4(everyone). Analysis and Evaluation of model performance/model comparison

##### 5(everyone). Lessons learned/things we would do differently/things we would do if we had more time 

# Everyone should be working in a single notebook. 

# Data pre-processing/filtering/cleaning

In [1]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re

import matplotlib.pyplot as plt

import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder

from textblob import TextBlob

books_file_path = 's3://cap4770-2024summer12/books_data.csv'
book_ratings_path = 's3://cap4770-2024summer12/Books_rating.csv'

In [3]:
books = pd.read_csv(books_file_path, usecols=["Title", "authors", "categories", "ratingsCount", "publishedDate"])
books.rename(columns={'Title': 'title', 'publishedDate': 'date_published', 'ratingsCount': 'ratings_count'}, inplace=True)

books["date_published"] = books["date_published"].str.strip()
books["date_published"] = books["date_published"].apply(lambda x: None if re.search(r'\d{4}', str(x)) is None else re.search(r'\d{4}', str(x)).group(0))
books = books.dropna()

books['date_published'] = books['date_published'].astype(int)
books = books[books["ratings_count"] >= 20]

books.head(5)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,title,authors,date_published,categories,ratings_count
111,The Rabbi's Cat,['Joann Sfar'],2005,['Comics & Graphic Novels'],25.0
269,The Castle in the Attic,['Elizabeth Winthrop'],2012,['Juvenile Fiction'],21.0
368,Jean Paul Sartres No Exit and the Flies,['Jean-Paul Sartre'],1976,['Drama'],22.0
475,The Gods of Mars,['Edgar Rice Burroughs'],2020,['Fiction'],26.0
520,A Wise Woman,['Philippa Gregory'],1994,['Fiction'],21.0


In [4]:
book_ratings = pd.read_csv(book_ratings_path, usecols=["User_id", "Title", "Price", "review/score", "review/text", "review/summary"])
book_ratings.rename(columns={'User_id': 'user_id', 'Title': 'title', "Price": "price", 'review/score': 'rating', 'review/text': 'review_text', 'review/summary': 'review_summary'}, inplace=True)
book_ratings = book_ratings.dropna()

book_ratings.head(5)

,title,price,user_id,rating,review_summary,review_text
10,Wonderful Worship in Smaller Churches,19.40,AZ0IOBU20TBOP,5.0,Outstanding Resource for Small Church Pastors,"I just finished the book, &quot;Wonderful Wors..."
11,Wonderful Worship in Smaller Churches,19.40,A373VVEU6Z9M0N,5.0,Small Churches CAN Have Wonderful Worship,Many small churches feel like they can not hav...
12,Wonderful Worship in Smaller Churches,19.40,AGKGOH65VTRR4,5.0,Not Just for Pastors!,I just finished reading this amazing book and ...
13,Wonderful Worship in Smaller Churches,19.40,A3OQWLU31BU1Y,5.0,Small church pastor? This is the book on worship,I hadn't been a small church pastor very long ...
14,Whispers of the Wicked Saints,10.95,A3Q12RK71N74LB,1.0,not good,I bought this book because I read some glowing...


In [5]:
user_ratings_sum = book_ratings.groupby('user_id')['rating'].sum()
user_ratings_sum = user_ratings_sum.to_frame('sum_of_user_ratings')

In [6]:
previous_ratings_submitted = book_ratings.groupby('user_id')['rating'].size()
previous_ratings_submitted = previous_ratings_submitted.to_frame('previous_ratings_submitted')
previous_ratings_submitted['previous_ratings_submitted'] = previous_ratings_submitted['previous_ratings_submitted'] - 1
previous_ratings_submitted.head(5)

,previous_ratings_submitted
user_id,
A00117421L76WVWG4UX95,0
A0015610VMNR0JC9XVL1,0
A00264602WCXBHHFPLTQ4,0
A00290423P2GEY37XWVAW,0
A0047322388NOTO4N8SKD,0


In [7]:
merged_book_attributes = books.merge(book_ratings, on='title', how='inner')\
                              .drop_duplicates(subset=['title', 'user_id'], keep=False)
                                
merged_book_attributes = merged_book_attributes[['user_id', 'date_published', 'ratings_count', 'price', 'review_summary', 'review_text', 'rating']]

merged_book_attributes['full_review_sentiment_score'] = merged_book_attributes['review_text'].apply(lambda review: TextBlob(review).sentiment.polarity)
merged_book_attributes['review_summary_sentiment_score'] = merged_book_attributes['review_summary'].apply(lambda review_summary: TextBlob(review_summary).sentiment.polarity)

merged_book_attributes.drop('review_summary', axis=1, inplace=True)
merged_book_attributes.drop('review_text', axis=1, inplace=True)

merged_book_attributes.head()

,user_id,date_published,ratings_count,price,rating,full_review_sentiment_score,review_summary_sentiment_score
0,A3F7XLGCTTUMWP,2005,25.0,16.93,4.0,0.130353,0.50
1,AGKW1Q9IUWIMT,2005,25.0,16.93,5.0,0.186603,0.65
2,A3NLCX4GEWUFLQ,2005,25.0,16.93,5.0,0.169697,0.80
3,A38P1LVVPBEEB4,2005,25.0,16.93,5.0,0.173779,0.60
4,AOTRWWEIYUFGN,2005,25.0,16.93,5.0,0.126473,0.00


In [8]:
merged_book_attributes = merged_book_attributes.merge(user_ratings_sum, on='user_id', how='left')
merged_book_attributes = merged_book_attributes.merge(previous_ratings_submitted, on='user_id', how='left')


merged_book_attributes['average_of_previous_ratings'] = (merged_book_attributes['sum_of_user_ratings'] - merged_book_attributes['rating']) / merged_book_attributes['previous_ratings_submitted']
merged_book_attributes['average_of_previous_ratings'] = merged_book_attributes['average_of_previous_ratings'].fillna(book_ratings['rating'].mean())
merged_book_attributes.head(10)

,user_id,date_published,ratings_count,price,rating,full_review_sentiment_score,review_summary_sentiment_score,sum_of_user_ratings,previous_ratings_submitted,average_of_previous_ratings
0,A3F7XLGCTTUMWP,2005,25.0,16.93,4.0,0.130353,0.500000,42.0,8,4.750000
1,AGKW1Q9IUWIMT,2005,25.0,16.93,5.0,0.186603,0.650000,5.0,0,4.240394
2,A3NLCX4GEWUFLQ,2005,25.0,16.93,5.0,0.169697,0.800000,5.0,0,4.240394
3,A38P1LVVPBEEB4,2005,25.0,16.93,5.0,0.173779,0.600000,5.0,0,4.240394
4,AOTRWWEIYUFGN,2005,25.0,16.93,5.0,0.126473,0.000000,43.0,11,3.454545
5,AUTBHG6070SL4,2005,25.0,16.93,4.0,0.045833,1.000000,440.0,107,4.074766
6,A80DUYOZPQ1A5,2005,25.0,16.93,5.0,0.151593,0.491667,5.0,0,4.240394
7,A310NXC0KH9N3T,2005,25.0,16.93,5.0,0.458333,0.000000,5.0,0,4.240394
8,A2LRXPITYJJINN,2005,25.0,16.93,5.0,0.318056,1.000000,5.0,0,4.240394
9,A25E0H37N21UEZ,2005,25.0,16.93,5.0,0.423077,0.462121,5.0,0,4.240394


In [70]:
X_train, X_test, y_train, y_test = train_test_split(merged_book_attributes[['ratings_count', 'price', 'full_review_sentiment_score', 'review_summary_sentiment_score', 'average_of_previous_ratings']], merged_book_attributes[['rating']], test_size=0.2, random_state=42)

# Linear regression

In [71]:
regr = LinearRegression()

In [72]:
regr.fit(X_train, y_train)

LinearRegression()

In [73]:
ratings_y_pred = regr.predict(X_test)

In [74]:
print(f"Coefficients: {regr.coef_}")
print(f"Mean squared error: {mean_squared_error(y_test, ratings_y_pred)}")
print(f"Coefficient of determination: {r2_score(y_test, ratings_y_pred)}")

Coefficients: [[-1.49247192e-05 -5.73777623e-04  1.69918669e+00  7.65049953e-01
   3.43600440e-01]]
Mean squared error: 1.1249582032893901
Coefficient of determination: 0.21447872346147234


In [75]:
# plt.scatter(X_test, y_test, color="black")
# plt.xlabel('Book Price')
# plt.ylabel('Rating')
# plt.plot(X_test, ratings_y_pred, color="blue", linewidth=3)

# K-nearest neighbors 

In [76]:
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [77]:
neigh.score(X_test, y_test)

0.02932358680826852

In [78]:
knn = KNeighborsRegressor()
param_grid = {'n_neighbors': range(1, 17), 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best r2 score:", grid_search.best_score_)

Best parameters: {'n_neighbors': 16, 'weights': 'uniform'}
Best r2 score: 0.2647600846596629
